## Kaggle data management api

In [ ]:
%%time
import os, sys

# !pip install kaggle # now download the api token and store it to /home/ec2-user/.kaggle/kaggle.json
# !cd /home/ec2-user/SageMaker
# !mkdir /home/ec2-user/.kaggle/
# !mv /home/ec2-user/SageMaker/kaggle.json /home/ec2-user/.kaggle/kaggle.json
# !chmod 600 /home/ec2-user/.kaggle/kaggle.json # for privacy
!df -h --total  # get the harddisk info
!nvcc --version  # Get the cuda version
os.chdir("/home/ec2-user/SageMaker")

In [10]:
# for root, dirs, files in os.walk("CervicalCancer_dataset", topdown = False):
#     print(len(files))
# #    for name in files:
# #       print(os.path.join(root, name))
#     for name in dirs:
#         print(os.path.join(root, name))

In [2]:
### running for setting up the git repository
!git clone https://github.com/RamsteinWR/IntelCervicalCancer.git
# !rm -rf /home/ec2-user/SageMaker/IntelCervicalCancer/data
# !mv /home/ec2-user/SageMaker/IntelCervicalCancerData /home/ec2-user/SageMaker/IntelCervicalCancer/data

# import shutil
# shutil.rmtree("/home/ec2-user/SageMaker/Haemorrhage_dataset", ignore_errors=True)
##### run this part for removing the repository
# !mv /home/ec2-user/SageMaker/IntelCervicalCancer/data /home/ec2-user/SageMaker/IntelCervicalCancerData
# !rm -rf /home/ec2-user/SageMaker/IntelCervicalCancer

# !pip install py7zr
# import py7zr
# with py7zr.SevenZipFile('CervicalCancer_dataset.7z', 'w') as archive:
#     archive.writeall('/home/ec2-user/SageMaker/CervicalCancer_dataset', 'base')

Cloning into 'IntelCervicalCancer'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 305 (delta 199), reused 224 (delta 121), pack-reused 0
Receiving objects: 100% (305/305), 1.53 MiB | 28.01 MiB/s, done.
Resolving deltas: 100% (199/199), done.


In [4]:
os.chdir("/home/ec2-user/SageMaker")

# Download the challenge data here 
# !kaggle competitions download -c intel-mobileodt-cervical-cancer-screening
# !unzip -o -q /home/ec2-user/SageMaker/intel-mobileodt-cervical-cancer-screening.zip -d /home/ec2-user/SageMaker/IntelCervicalCancer/data  #-q for quitely no verbose
# !rm -rf /home/ec2-user/SageMaker/intel-mobileodt-cervical-cancer-screening.zip

In [ ]:
# !mv /home/ec2-user/SageMaker/PneumoniaData/stage_2_detailed_class_info.csv /home/ec2-user/SageMaker/PneumoniaData/detailed_class_info.csv
# !mv /home/ec2-user/SageMaker/PneumoniaData/stage_2_test_images /home/ec2-user/SageMaker/PneumoniaData/test_dicoms

# !mv /home/ec2-user/SageMaker/IntelCervicalCancer/data/test_stg2/* /home/ec2-user/SageMaker/IntelCervicalCancer/data/test/
# !rm -rf IntelCervicalCancer/data/train1

# !pip install py7zr
# import py7zr
# with py7zr.SevenZipFile('/home/ec2-user/SageMaker/IntelCervicalCancer/data/test_stg2.7z', mode='r', password='byecervicalcancer') as z:
#     z.extractall()

# !zip -r -0 -q CervicalCancer_dataset.zip /home/ec2-user/SageMaker/CervicalCancer_dataset


## Preprocessing the authenication management

In [1]:
%%time
import boto3
import botocore
from botocore.exceptions import ClientError
from tqdm import tqdm
import os 
import urllib.request
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = "cervical-cancer-dataset"
region_name="us-east-1"
aws_access_key_id = "ahiguhgiuf"
aws_secret_access_key = "hagudfyguyagufygu"

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)    
    data = open(file, "rb")
    key = channel + '/' + file
    print("Uploading file {} to s3://{}/{}".format(file, bucket, channel))
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def upload_dir_to_s3(bucket, s3_folder, dir_to_upload):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
    print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_folder))
    # enumerate local files recursively
    for root, dirs, files in os.walk(dir_to_upload):
        for filename in tqdm(files):
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, dir_to_upload)
            s3_path = os.path.join(s3_folder, relative_path).replace("\\", "/")
            try:
                s3_client.head_object(Bucket=bucket, Key=s3_path)
                print("Path found on S3! Deleting %s..." % s3_path)
                try:
                    s3_client.delete_object(Bucket=bucket, Key=s3_path)
                    try:
#                         print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_path)
                        s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                    except ClientError as e:
                        logging.error(e)
                except:
                    print("Unable to delete from s3 %s..." % s3_path)
            except:
                try:
                    s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                except ClientError as e:
                    logging.error(e)
    print("Upload completed successfully.")
    
def download_dir(s3_folder, local_path, bucket=""):
    """
    params:
    - s3_folder: pattern to match in s3
    - local_path: local_path path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    client = boto3.client('s3', region_name=region_name)
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket': bucket,
        'Prefix': s3_folder,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local_path, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    print("{} files found in {} directories. Downloading now...".format(len(keys), len(dirs)))
    for k in tqdm(keys):
        dest_pathname = os.path.join(local_path, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        try:
#             print("Downloading {}".format(dest_pathname))
            client.download_file(bucket, k, dest_pathname)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
    print("{} files downloaded successfully.".format(len(keys)))
    
# download('http://data.lip6.fr/cadene/pretrained/home/ec2-user/SageMaker/rsna19/models/se_resnext50_32x4d-a260b3a4.pth')
# upload_to_s3("pretrained", 'se_resnext50_32x4d-a260b3a4.pth')
# upload_dir_to_s3(bucket=bucket, s3_folder='aptos-2019', dir_to_upload="/home/ec2-user/SageMaker/data")

# download_dir(s3_folder='aptos-2015/train_images_768/', local_path='/home/ec2-user/SageMaker/data/aptos-2015/train_images_768/', bucket=bucket)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


CPU times: user 2.8 s, sys: 3.44 s, total: 6.24 s
Wall time: 636 ms


### multipart uploader

In [2]:
%%time
import argparse
import os

import boto3


class S3MultipartUpload(object):
    # AWS throws EntityTooSmall error for parts smaller than 5 MB
    PART_MINIMUM = int(5e6)

    def __init__(self,
                 bucket,
                 key,
                 local_path,
                 aws_access_key_id,
                 aws_secret_access_key,
                 part_size=int(6e6),
                 profile_name=None,
                 region_name="us-east-1",
                 verbose=False): # keep it false unless it will verbose all connnectivity logs
        self.bucket = bucket
        self.key = key
        self.path = local_path
        self.total_bytes = os.stat(local_path).st_size
        self.part_bytes = part_size
#         assert part_size > self.PART_MINIMUM
#         assert (self.total_bytes % part_size == 0
#                 or self.total_bytes % part_size > self.PART_MINIMUM)
        self.s3 = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                        aws_secret_access_key=aws_secret_access_key,
                                        region_name=region_name).client("s3")
        if verbose:
            boto3.set_stream_logger(name="botocore")

    def abort_all(self):
        mpus = self.s3.list_multipart_uploads(Bucket=self.bucket)
        aborted = []
        print("Aborting", len(mpus), "uploads")
        if "Uploads" in mpus:
            for u in mpus["Uploads"]:
                upload_id = u["UploadId"]
                aborted.append(
                    self.s3.abort_multipart_upload(
                        Bucket=self.bucket, Key=self.key, UploadId=upload_id))
        return aborted

    def create(self):
        mpu = self.s3.create_multipart_upload(Bucket=self.bucket, Key=self.key)
        mpu_id = mpu["UploadId"]
        return mpu_id

    def upload(self, mpu_id):
        parts = []
        uploaded_bytes = 0
        with open(self.path, "rb") as f:
            i = 1
            while True:
                data = f.read(self.part_bytes)
                if not len(data):
                    break
                part = self.s3.upload_part(
                    Body=data, Bucket=self.bucket, Key=self.key, UploadId=mpu_id, PartNumber=i)
                parts.append({"PartNumber": i, "ETag": part["ETag"]})
                uploaded_bytes += len(data)
                print("{0} of {1} uploaded ({2:.3f}%)".format(
                    uploaded_bytes, self.total_bytes,
                    as_percent(uploaded_bytes, self.total_bytes)))
                i += 1
        return parts

    def complete(self, mpu_id, parts):
        result = self.s3.complete_multipart_upload(
            Bucket=self.bucket,
            Key=self.key,
            UploadId=mpu_id,
            MultipartUpload={"Parts": parts})
        return result


# Helper
def as_percent(num, denom):
    return float(num) / float(denom) * 100.0


def parse_args():
    parser = argparse.ArgumentParser(description='Multipart upload')
    parser.add_argument('--bucket', required=True)
    parser.add_argument('--key', required=True)
    parser.add_argument('--path', required=True)
    parser.add_argument('--region', default="us-east-1")
    parser.add_argument('--profile', default=None)
    return parser.parse_args()


bucket = "cervical-cancer-dataset"
region_name = "us-east-1"
aws_access_key_id = "haidhfaysdguyf"
aws_secret_access_key = "huidhfoiuahdfuha"


def multipart_upload(key, local_path):
    # args = parse_args()
    # mpu = S3MultipartUpload(
    #     args.bucket,
    #     args.key,
    #     args.path,
    #     profile_name=args.profile,
    #     region_name=args.region)
    mpu = S3MultipartUpload(
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        bucket=bucket,
        key=key,
        local_path=local_path,
        region_name='us-east-1')
    # abort all multipart uploads for this bucket (optional, for starting over)
    mpu.abort_all()
    # create new multipart upload
    mpu_id = mpu.create()
    # upload parts
    parts = mpu.upload(mpu_id)
    # complete multipart upload
    print(mpu.complete(mpu_id, parts))
# if __name__ == "__main__":
#     main()

# multipart_upload(key='CervicalCancer_dataset/dataset_in_numpy', local_path='/home/ec2-user/SageMaker/CervicalCancer_dataset/dataset_in_numpy/X_train.npy')

CPU times: user 17 µs, sys: 19 µs, total: 36 µs
Wall time: 40.5 µs


## Uploading files to s3

<a id='idg4c.1'></a>
## Dependencies
___
### import packages and check SageMaker version

In [3]:
# !pip install -r /home/ec2-user/SageMaker/IntelCervicalCancer/requirements.txt
# !rm -rf CervicalCancer_dataset/detect_data

In [1]:
import json
import torch
import tarfile
import pickle
import matplotlib.pyplot as plt
import torchvision as tv
import pathlib                          # Path management tool (standard library)
import subprocess                       # Runs shell commands via Python (standard library)
import sagemaker                        # SageMaker Python SDK
from sagemaker.pytorch import PyTorch   # PyTorch Estimator for TensorFlow

from sagemaker.s3 import S3Uploader

# S3Uploader.upload(local_path='/home/ec2-user/SageMaker/CervicalCancer_dataset/dataset_in_numpy/X_train.npy', 
#                   desired_s3_uri="S3://cervical-cancer-dataset/CervicalCancer_dataset/dataset_in_numpy/")

## Training the simple model
### adding some modules to path

In [2]:
import sys, os
sys.path.append("/home/ec2-user/SageMaker/IntelCervicalCancer")
sys.path.append("/home/ec2-user/SageMaker/IntelCervicalCancer/src")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/models/DeformableConvNets")
sys.path.append("/home/ec2-user/SageMaker/IntelCervicalCancer/src/kfold_train")
sys.path.append("/home/ec2-user/SageMaker/IntelCervicalCancer/src/scripts")
print(sys.path)

['', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python36.zip', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/lib-dynload', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ec2-user/.ipython', '/home/ec2-user/SageMaker/IntelCervicalCancer', '/home/ec2-user/SageMaker/IntelCervicalCancer/src', '/home/ec2-user/SageMaker/PneumoniaRSNA1/models/DeformableConvNets', '/home/ec2-user/SageMaker/IntelCervicalCancer/src/kfold_train', '/home/ec2-user/SageMaker/IntelCervicalCancer/src/scripts']


In [5]:
# import os, sys
os.chdir("/home/ec2-user/SageMaker/IntelCervicalCancer/")

# !python bbox_data_preproc.py        

Loaded train_val.pickle from /home/ec2-user/SageMaker/CervicalCancer_dataset/train_val.pickle
Augmenting the data: Output being saved to /home/ec2-user/SageMaker/CervicalCancer_dataset/detect_data/data001_size512/train
979it [08:27,  1.93it/s]
Saved augmented data to /home/ec2-user/SageMaker/CervicalCancer_dataset/detect_data/data001_size512/train
Augmenting the data: Output being saved to /home/ec2-user/SageMaker/CervicalCancer_dataset/detect_data/data001_size512/val
485it [04:24,  1.83it/s]
Saved augmented data to /home/ec2-user/SageMaker/CervicalCancer_dataset/detect_data/data001_size512/val


In [6]:
import os, sys
os.chdir("/home/ec2-user/SageMaker/IntelCervicalCancer/")

# !python train_detector.py

14784it [01:36, 152.53it/s]
6880it [00:45, 152.48it/s]
Traceback (most recent call last):
  File "train_detector.py", line 77, in <module>
    N_max=15616)  # total 15664 here 15616/64=244
  File "/home/ec2-user/SageMaker/IntelCervicalCancer/src/scripts/bbox_data_preproc.py", line 179, in load_dataset
    np.save(join(dataset_in_numpy, 'X_train.npy'), X_train)
  File "<__array_function__ internals>", line 6, in save
  File "/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/numpy/lib/npyio.py", line 524, in save
    file_ctx = open(file, "wb")
FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/CervicalCancer_dataset/dataset_in_numpy/X_train.npy'


In [18]:
hyperparameters = {
    'epochs': 10, 
    'batch-size': 64, 
    'learning-rate': 0.001,
    'workers': 4
}

In [19]:
estimator_config = {
    'entry_point': 'train_detector.py',
    'source_dir': '/home/ec2-user/SageMaker/IntelCervicalCancer',
    'framework_version': '1.6.0',
    'py_version': 'py3',
    'instance_type': 'ml.p2.8xlarge',
    'instance_count': 1,
    'volume_size':100,
    'checkpoint_local_path': None,
    'max_run':86400,
    'role': sagemaker.get_execution_role(),
    'output_path': 's3://cervical-cancer-dataset/training_job',
    'hyperparameters': hyperparameters
}

In [20]:
pytorch_estimator = PyTorch(**estimator_config)

In [21]:
data_channels = {
    'dataset_in_numpy': 's3://cervical-cancer-dataset/CervicalCancer_dataset/dataset_in_numpy',
}

In [22]:
pytorch_estimator.fit(data_channels)

2021-01-04 11:45:29 Starting - Starting the training job...
2021-01-04 11:45:53 Starting - Launching requested ML instancesProfilerReport-1609760728: InProgress
............
2021-01-04 11:47:54 Starting - Preparing the instances for training.........
2021-01-04 11:49:15 Downloading - Downloading input data..................................................................
2021-01-04 12:00:36 Training - Downloading the training image...
2021-01-04 12:01:04 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-04 12:00:53,893 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-01-04 12:00:53,970 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-04 12:01:00,200 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-04 12:01:01,128 sagemaker

UnexpectedStatusException: Error for Training job pytorch-training-2021-01-04-11-45-28-363: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python train_detector.py --batch-size 64 --epochs 10 --learning-rate 0.001 --workers 4"
Traceback (most recent call last):
  File "train_detector.py", line 77, in <module>
    N_max=15616)  # total 15664 here 15616/64=244
  File "/opt/ml/code/src/scripts/bbox_data_preproc.py", line 171, in load_dataset
    X_train = np.load(join(dataset_in_numpy, 'X_train.npy'))
  File "/opt/conda/lib/python3.6/site-packages/numpy/lib/npyio.py", line 440, in load
    pickle_kwargs=pickle_kwargs)
  File "/opt/conda/lib/python3.6/site-packages/numpy/lib/format.py", line 771, in read_array
    array.shape = shape
ValueError: cannot reshape array of size 10737419232 into shape (14784,3,512,512)